In [1]:
from bs4 import BeautifulSoup
from pprint import pprint
import requests
import urllib.request
import os
import urllib.parse
import shutil
from io import BytesIO

#import cv2
from PIL import Image
import time
import tensorflow.keras
from PIL import Image, ImageOps
import numpy as np


In [19]:
fakeBlogKeywordList = ["스토리앤","seoulouba","mateb.kr","revu","weble","ohmyblog","mrblog","tble",
    "dinnerqueen","%EA%B3%B5%EC%A0%95%EA%B1%B0%EB%9E%98%EC%9C%84%EC%9B%90%ED%9A%8C-%EB%AC%B8%EA%B5%AC",
    "%EB%B8%94%EB%A1%9C%EA%B7%B8%EC%9B%90%EC%A0%95%EB%8C%80","%EA%B3%B5%EC%A0%95%EC%9C%84","%EA%B3%B5%EC%A0%95%EB%AC%B8%EA%B5%AC"
    "banner_","%EC%9D%B4%EC%8A%88%EB%B8%94%EB%A1%9C%EA%B7%B8","쉬즈블로그","%EC%89%AC%EC%A6%88%EB%B8%94%EB%A1%9C%EA%B7%B8",
    "%EB%B0%B0%EB%84%88","http://echoblog.net/images/sponsor-banner.png","%EB%AC%B4%EC%83%81%EC%A0%9C%EA%B3%B5",
                      "sponsor","banner","echoblog","reviewplace","www.storyn.kr"]

In [3]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import os
import numpy as np
import matplotlib.pyplot as plt

#define const value
batch_size = 6
epochs = 10
IMG_HEIGHT = 120
IMG_WIDTH = 240


In [4]:
def create_model():
    model = Sequential([
        Conv2D(16, 3, padding='same', activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH ,3)),
        MaxPooling2D(),
        Conv2D(32, 3, padding='same', activation='relu'),
        MaxPooling2D(),
        Conv2D(64, 3, padding='same', activation='relu'),
        MaxPooling2D(),
        Flatten(),
        Dense(512, activation='relu'),
        Dense(1)
    ])
    model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])
    return model

In [21]:
#load model
checkpoint_path = "model_0720_v2(batch-4)/cp-0010.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

latest = tf.train.latest_checkpoint(checkpoint_dir)
print(latest)

# create model
model = create_model()
model.load_weights(latest)

data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)
np.set_printoptions(suppress=True)

model_0720_v2(batch-4)/cp-0010.ckpt


In [6]:
def convertToJpeg(im):
    with BytesIO() as f:
        new_img = im.convert("RGB")
        new_img.save(f, format='JPEG')
        return f.getvalue()


In [7]:
def PredictionWithoutDownload(uri):

    start_time = time.time();
    filename = uri.split('/')[-1].split('?')[0];
    img_url2 = urllib.parse.quote_plus(str(filename))
    img_url_final = uri.replace(uri.split('/')[-1].split('?')[0],img_url2)
    
#     response = requests.get(uri)
    response = urllib.request.urlopen(img_url_final).read()
#     print("---[1] {}s seconds---".format(time.time()-start_time,".2f"))
    start_time = time.time();
    #image = tf.keras.preprocessing.image.load_img(image_path,target_size=(IMG_HEIGHT, IMG_WIDTH))
#     img = Image.open(BytesIO(response.content))
    img = Image.open(BytesIO(response))
    
    if(img.format == "PNG") or (img.format == "GIF"):
        print("this image is",img.format,"type. could be change to jpg")
        png_to_jpg = convertToJpeg(img)
        img = Image.open(BytesIO(png_to_jpg))
#         img.show()
#     print("---[2] {}s seconds---".format(time.time()-start_time,".2f"))
    start_time = time.time();
    #width,height
    resize_img = img.resize((IMG_WIDTH, IMG_HEIGHT))
#     print("---[3] {}s seconds---".format(time.time()-start_time,".2f"))
    start_time = time.time();

    input_arr = tf.keras.preprocessing.image.img_to_array(resize_img)
    input_arr = np.array([input_arr])  # Convert single image to a batch.
#     print("---[4] {}s seconds---".format(time.time()-start_time,".2f"))
    start_time = time.time();
    
    predictions_class = model.predict_classes(input_arr)
#     print("---[5] {}s seconds---".format((time.time()-start_time),".2f"))

    print("========pridiction[Without Download]========");
#     print(test_data_gen.class_indices)
    print("predictions class : " , predictions_class);
    if(predictions_class == [[0]]):
        predictions = model.predict(input_arr)
        print("predictions : ",predictions);

        print("this is fake blog")
        saveImageVol2(uri,predictions)
#     print("predictions : " , predictions)
    return predictions_class


In [8]:
# =====================================================================
#이미지 저장만을 위한 함수
def saveImageVol2(img_url,predict):
    list = os.listdir("ad_img")
    last_index = len(list)
#         print("last_index : ",last_index)
    dirPath = 'ad_img'
    imgPath = f'ad_img/{last_index}_{predict}.jpg'
    
    if not os.path.isdir(dirPath):
        #폴더 생성
        try:
            os.makedirs(dirPath)
        except Exception as ex:
            print("error catch : " , ex)
    
    try:
        filename = img_url.split('/')[-1].split('?')[0];
        img_url2 = urllib.parse.quote_plus(str(filename))
        img_url_final = img_url.replace(img_url.split('/')[-1].split('?')[0],img_url2)
        urllib.request.urlretrieve(img_url_final, imgPath)
        #분류
        #{'advertise': 0, 'image': 1}
        #my_prediction  = teachableMachinePrediction(imgPath);
#         print("prediction : ", my_prediction[0]);
    except Exception as ex:
        print("img save error : " , ex)
        return False

In [22]:
# =====================================================================
#이미지 분류 [my model]

def imageClassification(image_path):
    image = tf.keras.preprocessing.image.load_img(image_path,target_size=(IMG_HEIGHT, IMG_WIDTH))
    input_arr = tf.keras.preprocessing.image.img_to_array(image)
#    print("input_arr type : " ,type(input_arr))
    input_arr = np.array([input_arr])  # Convert single image to a batch.

    predictions_class = model.predict_classes(input_arr)
    predictions = model.predict(input_arr)

    print("========pridiction========");
#     print(test_data_gen.class_indices)
    print("predictions class : " , predictions_class);
    print("predictions : " , predictions);
#    print("predictions argmax : ",np.argmax(predictions[0])); # same way
    return predictions_class,predictions;

# =====================================================================
#이미지 저장
def saveImage(div_option,index,img_url,nickName,postNumber):
    #블로그 유저 별로 폴더를 나눌건지에 대한 옵션
    #true -> path : image/{nickName}/{postNumber}
    #false -> path : image/myimg/
    if(div_option):
        if nickName is "":
            nickName = "unknown"
        if postNumber is "":
            postNumber = "1"        
        imgPath = f'image/{nickName}/{postNumber}/{index}.jpg'
        dirPath = f'image/{nickName}/{postNumber}'
    #폴더별로 분기하지 않을떄 path : image/myimg/
    else:
        list = os.listdir("image/temp")
        last_index = len(list)
#         print("last_index : ",last_index)
        dirPath = 'image/temp'
        imgPath = f'image/temp/{last_index}.jpg'
    
    if not os.path.isdir(dirPath):
        #폴더 생성
        try:
            os.makedirs(dirPath)
        except Exception as ex:
            print("error catch : " , ex)
                
    #파일 저장
    #temp에 저장 -> classification -> 분류 저장
    
    try:
        filename = img_url.split('/')[-1].split('?')[0];
        img_url2 = urllib.parse.quote_plus(str(filename))
        img_url_final = img_url.replace(img_url.split('/')[-1].split('?')[0],img_url2)
#         print("img_url_final : " ,img_url_final);
        for item in fakeBlogKeywordList:
            if(img_url_final.find(item) > 0):
                print('this is fake img')

                imgPath = f"image/temp/fake_{last_index}.jpg"
                break;
        
        urllib.request.urlretrieve(img_url_final, imgPath)
        #분류
        #{'advertise': 0, 'image': 1}
        my_prediction, score  = imageClassification(imgPath);
        #my_prediction  = teachableMachinePrediction(imgPath);
#         print("prediction : ", my_prediction[0]);
        
        if (my_prediction[0] == 0):
            list = os.listdir("ad_img")
            last_index = len(list)
            #os.rename(imgPath, f'ad_img/{last_index}.jpg')
            #shutil.move(imgPath, f'ad_img/{last_index}.jpg')
            if(imgPath.find("fake_") > 0):  #모델, url 둘다 이미지로 판단
                os.replace(imgPath, f'ad_img/url_model_{last_index}_{score}.jpg')
            else: #모델만 이미지로 판단
                os.replace(imgPath, f'ad_img/model_{last_index}_{score}.jpg')
                with open("fake_img_url_list.txt", "a") as file:
                    file.write(f'model_{last_index}_{score}.jpg\n')
                    file.write(f'{img_url_final}\n')
                    file.write("================================\n")
            return False
        else: # url만 이미지로 판단
            list = os.listdir("not_fake")
            last_index = len(list)
            if(imgPath.find("fake_") > 0):
                list = os.listdir("ad_img")
                last_index = len(list)
                os.replace(imgPath, f'not_fake/url_{last_index}_{score}.jpg')
            else: #모델, url 둘다 이미지로 판단
                if(score <= 1500):  #confidence가 1500이하일때만 저장 
                    os.replace(imgPath, f'not_fake/url_model_{last_index}_{score}.jpg')
        return True
    except Exception as ex:
        print("img save error : " , ex)
        return False
    
    
# =====================================================================
# image gethering
def findBlogImgGethering(searchQuery, searchOption, page):
    try:
        start_time = time.time();

        url = f'https://search.naver.com/search.naver?query={searchQuery}&sm=tab_pge&srchby=all&st={searchOption}&where=post&start={page}'
        html = requests.get(url)
        # 1차, blog URL 찾기
        soup = BeautifulSoup(html.text, 'html.parser')
        index = 0;

        url_list = [];
        for li_item in soup.find_all('li',{'class' : 'sh_blog_top'}):
            child_item = li_item.find('a',{'class' : 'sh_blog_title'})
            title = child_item.attrs['title']
            href = child_item.attrs['href']

            #url list 저장
            url_list.append(href)

            print(f'title : {title} , href : {href}')
            print('----------------------------------')

        # url 파싱 및 예외처리
        for url_item in url_list:
            nickName = "";
            postNumber = "";

            if(url_item.find("blog.me") > 0):
                parsing = url_item;
                parsing = parsing.replace("https://","")
                nickName = parsing.split('.')[0]
                postNumber = parsing.split('/')[1]
                blogUrl = "https://m.blog.naver.com/" + nickName + "?Redirect=Log&logNo=" + postNumber

            else:
                blogUrl = url_item.replace("https://", "https://m.");

            nickName = blogUrl.split('/')[-1].split('?')[0]
            postNumber = blogUrl.split('/')[-1].split('?')[1].split('=')[-1]

            if nickName is "":
                nickName = "unknown"
            if postNumber is "":
                postNumber = "1"

            print("------ digging more -------")
            print("nickName : " , blogUrl.split('/')[-1].split('?')[0])
            print("postNumber : " , blogUrl.split('/')[-1].split('?')[1].split('=')[-1])
            print("blogUrl : " , blogUrl)        
            start_time = time.time();
            #전체 목록 순회
            blog_html = requests.get(blogUrl)
            blog_soup = BeautifulSoup(blog_html.text, 'html.parser')
            blog_image_class = blog_soup.find_all('div',{'class' : 'se-image'});

            for div_obj in blog_image_class:
                for idx, img_item in enumerate(div_obj.find_all('img')):
                    img_url = img_item.attrs['src'];

                    if(img_url.find("w80_blur") > 0):
                        img_url = img_url.replace("w80_blur" , "w800")
                        #np array after save image(for crawling data)
                    isSaveSuccess = saveImage(False,index,img_url,nickName,postNumber)
                    #np arr from img_url for validation
#                     PredictionWithoutDownload(img_url)
                    if(isSaveSuccess):  
                        index = index+1
    #         print("---{}s seconds---".format(time.time()-start_time),".2f")
    except Exception as ex:
        print("error catch : " , ex)
        print("url : " ,img_url)


In [24]:
searchQuery = "무선이어폰 추천"
searchOption = 'sim'  #sim or date  
page = 1
end_page = 10
for i in list(range(page,end_page)):
    if i>1:
        page = page + 10;
    findBlogImgGethering(searchQuery,searchOption, page)
# findBlogImgGethering(searchQuery,searchOption,page)

title : 무선이어폰 추천 블루투스 5.0 가성비 까지 잡았다 , href : https://blog.naver.com/armeise5?Redirect=Log&logNo=222005588082
----------------------------------
title : 엠포우 무선 이어폰, MPOW T6 사용기와 추천 포인트 , href : https://blog.naver.com/copy98?Redirect=Log&logNo=222025997007
----------------------------------
title : 가성비 무선이어폰 추천 순위 따져보니 O2-PODS , href : https://hyunjung5340.blog.me/222026095918
----------------------------------
title : 가성비 무선이어폰 추천 에어트랙 3.0의 매력 포인트는? , href : https://blog.naver.com/dogslife78?Redirect=Log&logNo=221985892775
----------------------------------
title : 블루투스 무선이어폰 추천! O2-PODS 가성비 갑이야! , href : https://blog.naver.com/lqud?Redirect=Log&logNo=222009594397
----------------------------------
title : 가성비 무선이어폰 추천하는 디스코즈 에어트랙 3.0 , href : https://sensefull7.blog.me/222004874667
----------------------------------
title : 가성비 무선이어폰 추천 코원 cx7 세븐팟 장단점 , href : https://blog.naver.com/strangefate?Redirect=Log&logNo=222033424736
----------------------------------
title : 가성비 무선이어폰 추천 디비누

========pridiction========
predictions class :  [[0]]
predictions :  [[-994.03815]]
========pridiction========
predictions class :  [[1]]
predictions :  [[503.23758]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-880.34247]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-437.0605]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-753.2333]]
========pridiction========
predictions class :  [[1]]
predictions :  [[3047.267]]
========pridiction========
predictions class :  [[1]]
predictions :  [[2941.0974]]
========pridiction========
predictions class :  [[1]]
predictions :  [[547.1965]]
========pridiction========
predictions class :  [[1]]
predictions :  [[76.62143]]
========pridiction========
predictions class :  [[1]]
predictions :  [[780.58777]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-1913.4689]]
------ digging more -------
nickName :  sensefull7
postNumber :  222004874667
blogUrl :

========pridiction========
predictions class :  [[0]]
predictions :  [[-174.02275]]
========pridiction========
predictions class :  [[1]]
predictions :  [[2339.9731]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-1287.5049]]
------ digging more -------
nickName :  zmsgfc
postNumber :  222032235159
blogUrl :  https://m.blog.naver.com/zmsgfc?Redirect=Log&logNo=222032235159
========pridiction========
predictions class :  [[1]]
predictions :  [[317.77255]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-295.7986]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-789.87683]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-512.5496]]
========pridiction========
predictions class :  [[1]]
predictions :  [[224.69627]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-592.4789]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-106.7439]]
========pridiction

========pridiction========
predictions class :  [[0]]
predictions :  [[-615.29346]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-22.844534]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-895.73145]]
========pridiction========
predictions class :  [[1]]
predictions :  [[1895.4111]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-96.57812]]
========pridiction========
predictions class :  [[1]]
predictions :  [[3922.2969]]
------ digging more -------
nickName :  moyeu119
postNumber :  221986298815
blogUrl :  https://m.blog.naver.com/moyeu119?Redirect=Log&logNo=221986298815
========pridiction========
predictions class :  [[1]]
predictions :  [[1648.1053]]
========pridiction========
predictions class :  [[1]]
predictions :  [[3530.1858]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-194.0503]]
========pridiction========
predictions class :  [[1]]
predictions :  [[330.2706]]
========pridict

========pridiction========
predictions class :  [[0]]
predictions :  [[-364.0378]]
========pridiction========
predictions class :  [[1]]
predictions :  [[1848.3063]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-178.62881]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-305.83084]]
========pridiction========
predictions class :  [[1]]
predictions :  [[259.59155]]
========pridiction========
predictions class :  [[1]]
predictions :  [[1629.6803]]
========pridiction========
predictions class :  [[1]]
predictions :  [[2055.68]]
========pridiction========
predictions class :  [[1]]
predictions :  [[3706.9246]]
========pridiction========
predictions class :  [[1]]
predictions :  [[3277.1804]]
========pridiction========
predictions class :  [[1]]
predictions :  [[250.9404]]
========pridiction========
predictions class :  [[1]]
predictions :  [[1005.6659]]
========pridiction========
predictions class :  [[1]]
predictions :  [[1683.6399]]
=====

========pridiction========
predictions class :  [[1]]
predictions :  [[3576.263]]
========pridiction========
predictions class :  [[1]]
predictions :  [[2203.8584]]
========pridiction========
predictions class :  [[1]]
predictions :  [[919.07855]]
========pridiction========
predictions class :  [[1]]
predictions :  [[1713.9673]]
========pridiction========
predictions class :  [[1]]
predictions :  [[3236.576]]
========pridiction========
predictions class :  [[1]]
predictions :  [[1366.9567]]
========pridiction========
predictions class :  [[1]]
predictions :  [[1983.195]]
========pridiction========
predictions class :  [[1]]
predictions :  [[775.0094]]
========pridiction========
predictions class :  [[1]]
predictions :  [[603.7067]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-267.5278]]
========pridiction========
predictions class :  [[1]]
predictions :  [[1996.6818]]
========pridiction========
predictions class :  [[0]]
predictions :  [[-934.6172]]
========p

UnboundLocalError: local variable 'img_url' referenced before assignment

In [11]:
# # blog gethering test
# def findBlogListTest(searchQuery, searchOption, page):
#     try:
#         print(f'\n###############{page}###############\n');
#         url = f'https://search.naver.com/search.naver?query={searchQuery}&sm=tab_pge&srchby=all&st={searchOption}&where=post&start={page}'
#         html = requests.get(url)
#         # 1차, blog URL 찾기
#         soup = BeautifulSoup(html.text, 'html.parser')
#         index = 0;

#         url_list = [];
#         for li_item in soup.find_all('li',{'class' : 'sh_blog_top'}):
#             child_item = li_item.find('a',{'class' : 'sh_blog_title'})
#             title = child_item.attrs['title']
#             href = child_item.attrs['href']

#             #url list 저장
#             url_list.append(href)

#             print(f'title : {title} , href : {href}')
#             print('----------------------------------')

#     except Exception as ex:
#         print("error catch : " , ex)
#         print("url : " ,img_url)

In [12]:
# searchQuery = "서울대입구역 하남돼지집"
# searchOption = 'sim'  #sim or date  
# page = 1
# end_page = 11
# # 1->11->21->순으로
# for i in list(range(page,end_page)):
#     if i>1:
#         page = page + 10;
#     findBlogListTest(searchQuery,searchOption,page)